In [ ]:
import pandas as pd
import numpy as np
import pandas_datareader.data as web
from collections import defaultdict
import matplotlib.pyplot as plt
from datetime import datetime  
import statsmodels.api as sm

# 信号前沿分析
pandas pandas_datareader.data  pdr.data.get_data_yahoo()  
pandas dataframe d.asfreq()  
pandas dataframe d.fillna()  
pandas dataframe d.head()  

In [ ]:
names  = ['AAPL','GOOG','MSFT','GS','MS','BAC','C']

df =web.get_data_yahoo(names,'2009/1/1','2012/6/1')
d1 = df['Adj Close']
d1.head()

In [ ]:
# pandas dataframe d.asfreq()
px = pd.DataFrame({n: d1[n] for n in names})
px = px.asfreq(freq='B').fillna(method='pad')
px.head() 

In [ ]:
'''
绘制每只股票的累计收益
pandas dataframe d.pct_change()
pandas dataframe d.add()
pandas dataframe d.cumprod()  
pandas dataframe d.sub()
pandas dataframe d.plot.line()
'''
rets = px.pct_change()
rets.add(1).cumprod().sub(1).plot.line()

In [ ]:
'''
对于投资组合的构建，我们要计算特定回顾期的动量，然后按降序排列并标准化  
python str printf-style String Formatting d
pandas dataframe d.shift() 
pandas dataframe d.rank() ascending 
pandas dataframe d.mean() 
pandas dataframe d.std()
pandas dataframe d.resample() 
pandas Resampling r.first()  
pandas Resampling r.apply() 
pandas dataframe d.mul()  
pandas dataframe d.add()  
pandas dataframe d.prod()
pandas series s.sub()
pandas series s.mean()
pandas series s.std()
numpy Mathematical functions np.sqrt() 
numpy Broadcasting 标量  
'''
def calc_mom(price,lookback,lag):
    mom_ret = price.shift(periods=lag).pct_change(periods=lookback)
    ranks =mom_ret.rank(axis=1,ascending=False)    
    demeaned = ranks.sub(ranks.mean(axis=1),axis=0)    
    return demeaned.div(demeaned.std(axis=1),axis=0)

compound = lambda x: x.add(1).prod().sub(1)
daily_sr =lambda x:x.mean()/x.std()

def strat_sr(prices,lb,hold):
    # 计算投资组合权重
    freq ='%dB' % hold
    port =calc_mom(prices,lb,lag=1)
    daily_rets = prices.pct_change()
    
    # 计算投资组合收益
    port = port.shift(periods=1).resample(rule=freq).first()
    returns =daily_rets.resample(rule=freq).apply(compound)
    port_rets = port.mul(returns).sum(axis=1)
    
    return daily_sr(port_rets)*np.sqrt(252/hold)

# 通过价格数据以及一对参数组合调用该函数将会得到一个标量值
strat_sr(px,70,30)

In [ ]:
'''
对参数网格应用strat_sr函数，并将结果保存在一个defaultdict中，最后将全部结果放进一个dataframe中
python Bulit-in Functions range()
python collections c.defaultdict()
pandas dataframe d.index
pandas Index i.name
pandas dataframe d.columns
matplotlib Pyplot function overview plt.figure()  
matplotlib figure f.add_subplot()  
matplotlib axes a.imshow() cmap/interpolation
matplotlib colors color.Colormap()  
matplotlib axes a.set_xlabel()  
matplotlib axes a.set_xticks()  
matplotlib axes a.set_xticklabels()
matplotlib axes a.set_ylabel()
matplotlib axes a.set_yticks()
matplotlib axes a.set_yticklabels()
matplotlib Pyplot function overview plt.colorbar()  
'''
lookbacks = range(20,90,5)
holdings = range(20,90,5)
dd = defaultdict(dict)
for lb in lookbacks:
    for hold in holdings:
        dd[lb][hold] = strat_sr(px,lb,hold)
ddf = pd.DataFrame(dd)
ddf.index.name = 'Holding Period'
ddf.columns.name = 'Lookback Period'

def heatmap(df,cmap=plt.cm.gray_r):
    fig = plt.figure()
    ax = fig.add_subplot(111)
    axim = ax.imshow(df.values,cmap=cmap,interpolation='nearest')
    ax.set_xlabel(df.columns.name)
    ax.set_xticks(np.arange(len(df.columns)))
    ax.set_xticklabels(list(df.columns))
    ax.set_ylabel(df.index.name)
    ax.set_yticks(np.arange(len(df.columns)))
    ax.set_yticklabels(list(df.index))
    plt.colorbar(mappable=axim)
    
heatmap(ddf)
print ("动量策略各种回顾期和持有期的夏普比率热图")

# 期货合约转仓  
用SPY交易所交易基金的部分价格作为标准普尔500指数的代理  
pandas dataframe d.loc[]  
pandas series s.mul()  

In [ ]:
px = web.get_data_yahoo('SPY','2011/8/1','2012/7/27').loc[:,'Adj Close'].mul(10)
px.head()

In [ ]:
'''
python datetime datetime()  
pandas series s.sort_values()
'''
expiry =  {'ESU2':datetime(2012,9,21),
          'ESZ2':datetime(2012,12,21)}
expiry = pd.Series(expiry).sort_values()
expiry

In [ ]:
'''
用Yahoo!Finance 的价格以及一个随机漫步和一些噪声来模拟这两份合约未来的走势：
numpy Random sampling (numpy.random) np.random.seed()  
numpy Random sampling (numpy.random) np.random.randint()
numpy The N-dimensional array (ndarray) ndarray.cumsum()  
numpy Array manipulation routines np.concatenate()
pandas dataframe d.values  
numpy Indexing Single element indexing 
numpy Broadcasting 同型矩阵  
pandas dataframe d.tail()  
'''
np.random.seed(12347)
N=200
walk=(np.random.randint(low=0,high=200,size=N)-100)*0.25
perturb =(np.random.randint(low=0,high=20,size=N) -10)*0.25
walk =walk.cumsum()

rng = pd.date_range(start=px.index[0],periods=len(px)+N,freq='B')
near = np.concatenate([px.values,px.values[-1]+walk])
far = np.concatenate([px.values,px.values[-1]+walk+perturb])
prices = pd.DataFrame({'ESU2':near,'ESZ2':far},index=rng)
prices.tail()

In [ ]:
def get_roll_weights(start,expiry,items,roll_periods=5):
    '''
    start : 用于计算加权矩阵的第一天
    expiry：由“合约代码 -> 到期日期” 组成的序列
    items：一组合约名称
    numpy Array creation routines np.zeros()
    pandas series s.iteritems()  
    python Built-in Functions enumerate()
    pandas Date offsets tseries.offsets.BusinessDay() 
    pandas General functions pd.date_range() start/end  
    numpy Array creation routines np.linspace() start/stop
    numpy Array creation routines np.linspace() num
    pandas series s.index  
    pandas index slice[]
    '''
    
    dates = pd.date_range(start,expiry[-1],freq='B')
    weights = pd.DataFrame(np.zeros((len(dates),len(items))),
                          index=dates,columns=items)
    prev_date = weights.index[0]
    for i,(item,ex_date) in enumerate(expiry.iteritems()):
        if i < len(expiry) -1 :
            weights.loc[prev_date:ex_date - pd.tseries.offsets.BusinessDay(),item] =1
            roll_rng = pd.date_range(end=ex_date-pd.tseries.offsets.BusinessDay(),
                                    periods=roll_periods + 1,freq='B')
            decay_weights = np.linspace(start=0,stop=1,num=roll_periods +1)
            
            weights.loc[roll_rng,item]=1 - decay_weights
            weights.loc[roll_rng,expiry.index[i+1]]= decay_weights
        else:
                weights.loc[prev_date:,item] =1
        prev_date = ex_date
    return weights

# 快到ESU2到期日的那几天的权重如下所示：
weights = get_roll_weights(start='2012/6/1',expiry=expiry,items=prices.columns)
weights.loc['2012-09-12':'2012-09-21']

In [ ]:
# pandas series s.sum() axis
rolled_returns = prices.pct_change().mul(weights).sum(axis=1)
rolled_returns.loc['2012-09-12':'2012-09-21']

# 移动相关系数与线性回归
首先从Yahoo! Finance加载一些价格序列  
pandas series s.pct_change()  
pandas series s.rolling()  
pandas Window Rolling.corr()  
pandas series s.plot.line()  
statsmodels api api

In [ ]:
aapl =d1.loc[:,'AAPL']
msft =d1.loc[:,'MSFT']
aapl_rets = aapl.pct_change()
msft_rets = msft.pct_change()
aapl_rets.rolling(window=250).corr(other=msft_rets).plot.line()

In [ ]:
'''
statsmodels ERROR MissingDataError: exog contains inf or nans
pandas Window Rolling.aggregate() *args  
pandas series s.loc[]
statsmodels OLS sm.OLS()  
statsmodels OLS OLS.fit()  
statsmodels RegressionResults RegressionResults.params
'''
def ols_model(aapl,msft):
    OLS = sm.OLS(endog=aapl,exog=msft.loc[aapl.index])
    model=OLS.fit()
    return model.params

beta =aapl_rets.rolling(window=250).aggregate(ols_model,msft_rets)
len(beta)

In [ ]:
beta.plot.line()